### Loading the libraries

In [137]:
import pandas as pd
import pyodbc
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
from dotenv import load_dotenv
load_dotenv()

from sqlalchemy import create_engine, text
import sqlalchemy


### Setting up the Database connection with MySQL

In [146]:
conn = pyodbc.connect(
    "DRIVER={MySQL ODBC 8.0 Unicode Driver};"
    f"SERVER={os.getenv('server')};"
    f"PORT={os.getenv('port')};"
    f"DATABASE={os.getenv('database')};"
    f"USER={os.getenv('user')};"
    f"PASSWORD={os.getenv('password')};"
    "OPTION=3;"
)

#conn.close()

#### Testing the DB connection

In [148]:
basic_query = "SELECT * FROM retail_sales"
retail_sales = pd.read_sql(basic_query, conn)

In [150]:
len(retail_sales), retail_sales.head(5)

(2000,
    transactions_id   sale_date sale_time  customer_id  gender   age  \
 0                1  2022-12-16  19:10:00           50    Male  34.0   
 1                2  2022-06-24  10:07:00          104  Female  26.0   
 2                3  2022-06-14  07:08:00          114    Male  50.0   
 3                4  2023-08-27  18:12:00            3    Male  37.0   
 4                5  2023-09-05  22:10:00            3    Male  30.0   
 
       category  quantity  price_per_unit   cogs  total_sale  
 0       Beauty       3.0            50.0   16.0       150.0  
 1     Clothing       2.0           500.0  135.0      1000.0  
 2  Electronics       1.0            30.0    8.1        30.0  
 3     Clothing       1.0           500.0  200.0       500.0  
 4       Beauty       2.0            50.0   24.0       100.0  )

In [154]:
### Trying SQLAlchemy
# # Define MySQL Connection String
# db_user = os.getenv("user")
# db_password = os.getenv("password")
# db_host = os.getenv('server')  # Change to your MySQL server if needed
# db_port = os.getenv('port')  # Default MySQL port
# db_name = os.getenv('database')  # Change this to your desired database name

# # Create SQLAlchemy Engine
# engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# # Test connection
# with engine.connect() as conn:
#     print("✅ Connected to MySQL successfully!")

# #Testing connection using pymysql 
# conn = pymysql.connect(db='sql_project_1', user='root', passwd='Mymummy1@', host='localhost', port=3306)
# conn.close()

### Exploratory Data Analysis

In [172]:
q0 = """SELECT * FROM retail_sales;"""
pd.read_sql(q0, conn)

,transactions_id,sale_date,sale_time,customer_id,gender,age,category,quantity,price_per_unit,cogs,total_sale
0,1,2022-12-16,19:10:00,50,Male,34.0,Beauty,3.0,50.0,16.0,150.0
1,2,2022-06-24,10:07:00,104,Female,26.0,Clothing,2.0,500.0,135.0,1000.0
2,3,2022-06-14,07:08:00,114,Male,50.0,Electronics,1.0,30.0,8.1,30.0
3,4,2023-08-27,18:12:00,3,Male,37.0,Clothing,1.0,500.0,200.0,500.0
4,5,2023-09-05,22:10:00,3,Male,30.0,Beauty,2.0,50.0,24.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,2023-12-07,18:49:00,2,Male,62.0,Clothing,1.0,50.0,25.0,50.0
1996,1997,2023-10-29,21:09:00,99,Male,52.0,Beauty,3.0,30.0,10.2,90.0
1997,1998,2023-10-20,19:53:00,90,Female,23.0,Beauty,4.0,25.0,24.0,100.0
1998,1999,2022-11-12,13:06:00,61,Female,36.0,Electronics,3.0,50.0,13.0,150.0


#### Finding the number of rows and columns in the SQL dataset

In [165]:
q1 = """ 
SELECT COUNT(*) as total_transactions FROM retail_sales ;
"""

pd.read_sql(q1, conn)

,total_transactions
0,2000


In [169]:
q2 = """ 
SELECT COUNT(*) AS column_count
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'retail_sales'
"""
pd.read_sql(q2, conn)

,column_count
0,11


- There are 2000 rows and 11 columns in the retail analysis dataset

### Let's explore the data further

In [178]:
#Finding the number of unique customers in the dataset and the division between male and female
q3 = """SELECT COUNT(DISTINCT customer_id) from retail_sales;"""
pd.read_sql(q3,conn)

,COUNT(DISTINCT customer_id)
0,155


- There are 155 unique customer IDs in the dataset

In [185]:
q4 = """
        SELECT DISTINCT customer_id, gender
        FROM retail_sales
        ORDER BY customer_id;
        """
pd.read_sql(q4, conn)

,customer_id,gender
0,1,Female
1,1,Male
2,2,Female
3,2,Male
4,3,Female
...,...,...
301,153,Male
302,154,Female
303,154,Male
304,155,Female


- However, when looked upon closely we can see that there are customer IDs having multiple genders 

In [196]:
#Finding the number of products sold category-wise
q5 = """ 
    SELECT gender, category, SUM(quantity) as total_products, SUM(total_sale) as amount_earned
    FROM retail_sales
    GROUP BY category, gender
    ORDER BY amount_earned DESC;
    """

pd.read_sql(q5, conn)

,gender,category,total_products,amount_earned
0,Female,Clothing,879.0,162460.0
1,Male,Electronics,820.0,160340.0
2,Female,Electronics,878.0,153470.0
3,Female,Beauty,829.0,149470.0
4,Male,Clothing,906.0,148610.0
5,Male,Beauty,706.0,137370.0


- From the above analysis we can see that the highest amount of revenue earned from Female customers was from 'Clothing' products and from Male customer was from 'Electronic' products 

### Analysis of Sales month wise

In [204]:
q6 = """
    SELECT DATE_FORMAT(sale_date, '%m') AS month, 
    SUM(total_sale) AS total_sales
    FROM retail_sales
    GROUP BY month
    ORDER BY month;
"""

pd.read_sql(q6, conn)

,month,total_sales
0,01,46425.0
1,02,41280.0
2,03,45035.0
3,04,50630.0
4,05,51990.0
5,06,45255.0
6,07,58120.0
7,08,49465.0
8,09,129330.0
9,10,126115.0


- We have data for 2 years, hence looking at 24 rows won't be conclusive enough. Instead looking at the total sales month-wise we can extrapolate that sales generally increase in the last 4 months of the year.